In [15]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import pickle

import numpy as np
import matplotlib.pyplot as plt

from utils import *
from autoencoder import *

In [2]:
'''
When taking inputs first check sanity and warn user.
ex: check file paths and output path (if output exists give warning before overwrite)
'''

'\nWhen taking inputs first check sanity and warn user.\nex: check file paths and output path (if output exists give warning before overwrite)\n'

In [21]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Device name:', torch.cuda.get_device_name(device), '\n')


Device name: NVIDIA A100-SXM4-80GB 



In [4]:
traj = read_traj(
    traj_ = '/orange/alberto.perezant/t.desilva/insulin/water/no_water_aligned/md_water.nc', 
    top_ = '/orange/alberto.perezant/t.desilva/insulin/water/no_water_aligned/system.parm7',
    stride = 1,
    memmap=False
)
n_atoms = traj.shape[2]
traj_dl = DataLoader(traj, batch_size=128, shuffle=True, drop_last=True, num_workers=4)


Trajectory stats : #_Frames = 100000	#_Atoms = 202
______________________________________________________________________ 

Start reading coordinates from trajectory to train model...
[100000 frames with stride 1]



oading trajectory: 100.00% |████████████████████|

In [5]:
model = AE(n_atoms=n_atoms)
model = LightAE(model=model, lr=1e-4, weight_decay=0)

In [6]:
print('Training Autoencoder:')
torch.set_float32_matmul_precision('medium')
trainer = pl.Trainer(max_epochs=3, accelerator='gpu', devices=1)
trainer.fit(model, traj_dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Training Autoencoder:


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4]

  | Name    | Type     | Params | Mode 
---------------------------------------------
0 | model   | AE       | 28.2 M | train
1 | loss_fn | RMSELoss | 0      | train
---------------------------------------------
28.2 M    Trainable params
0         Non-trainable params
28.2 M    Total params
112.708   Total estimated model params size (MB)
33        Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


Autoencoder training complete

______________________________________________________________________ 



In [ ]:
fig, ax = plt.subplots(figsize=(10,4), ncols=2)

ax[0].tick_params(axis='both', which='major', labelsize=12, labelfontfamily='monospace')
ax[0].plot(model.epoch_losses, color='red')
ax[0].set_xlabel('epoch', fontsize=14, fontfamily='serif')
ax[0].set_ylabel('RMSE $(nm)$', fontsize=14, fontfamily='serif')

ax[1].tick_params(axis='both', which='major', labelsize=12, labelfontfamily='monospace')
x = rmsd_(model=model, dl=traj_dl, 
                    top = '/orange/alberto.perezant/t.desilva/insulin/water/no_water_aligned/system.parm7')*10
bins = 100
ax[1].hist(x, bins=bins)

hist, bin_edges = np.histogram(x, bins=bins)
med = approx_median(hist=hist, bin_edges=bin_edges)

ax[1].set_xlabel(r'RMSD $(\AA)$', fontsize=14, fontfamily='serif')
ax[1].set_ylabel('Density', fontsize=14, fontfamily='serif')
ax[1].text(0.58, 0.95, f'AVG: {np.mean(x):.3f}\nSTD: {np.std(x):.3f}\nMED: {med[0]:.3f}\u00B1{med[1]:.3f}', transform=ax[1].transAxes, fontsize=12,
               verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax[1].axvline(x=np.mean(x), color='grey', linestyle='--', label='Vertical line at x=7')
#give average value and the standard deviation

plt.tight_layout()
plt.show()

calculating RMSD :   0.06% |                     |

In [ ]:
# are there ways to get correlation coefficient for 3d data? -- get it to validate model

In [ ]:
FitMetrics(model=model, dl=traj_dl, 
                    top = '/orange/alberto.perezant/t.desilva/insulin/water/no_water_aligned/system.parm7')[0]

In [ ]:
print('Saving model parmeters')
torch.save(model, 'model.pt')

In [ ]:
print('\nInitiate compression')
encoder = model.model.encoder.to(device)
traj_dl = DataLoader(traj, batch_size=128, shuffle=False, drop_last=False, num_workers=4)
# del model
z = [] # Latent
with torch.no_grad():
    encoder.eval()
    for batch in tqdm(traj_dl, desc="compressing "):
        batch = batch.to(device=device, dtype=torch.float32)
        z.append(encoder(batch))

pickle.dump(z, open("compressed_file.pkl", "wb"))
print('\n'+'_'*70)

In [ ]:
org_size = os.path.getsize('/orange/alberto.perezant/t.desilva/insulin/water/no_water_aligned/md_water.nc')
comp_size = os.path.getsize(f"compressed_file.pkl")
compression = 100*(1 - comp_size/org_size)

template = "{string:<20} :{value:15.3f}"
print(template.format(string='Original Size [MB]', value=round(org_size*1e-6,3)))
print(template.format(string='Compressed Size [MB]', value=round(comp_size*1e-6,3)))
print(template.format(string='Compression %', value=round(compression,3)))
print('---')

# save the rmsd, r2, mae beforehand with or without plots so we can acces to calculate following
print(template.format(string='RMSD [\u212B]', value=round(np.mean(x),3))) 

In [14]:
# Add a cleanup - remove memmap and unnecessary files generated in the process -- utils

In [ ]:
# Add the compressed file path